In [20]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

### Read from CSV dataframe method

In [15]:
"""more infor here
https://towardsdatascience.com/spark-essentials-how-to-read-and-write-data-with-pyspark-5c45e29227cd
"""
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load('sales_info.csv')
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



### transformation 

In [17]:
#transformation
"""Select count(1) from df where sales > 200 group by Company"""
group_df=df.where("Sales >200").groupBy("Company")
group_df.count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    3|
|   GOOG|    1|
|     FB|    2|
|   MSFT|    2|
+-------+-----+



### Spark SQL 

In [25]:
#SQL can be done only on a table or a view
# So Create a view
df.createOrReplaceTempView('sales_tbl')
agg_df = spark.sql("select company,count(1) from sales_tbl where sales > 200 group by company")
agg_df.show()

+-------+--------+
|company|count(1)|
+-------+--------+
|   APPL|       3|
|   GOOG|       1|
|     FB|       2|
|   MSFT|       2|
+-------+--------+



### Read csv and JSON and Parquet

In [33]:
df_csv=spark.read.format("csv").option("header","true").option("inferSchema","true").load('flight-time.csv')
print("CSV Schema" +df_csv.schema.simpleString())
df_csv.show(5)

CSV Schemastruct<FL_DATE:string,OP_CARRIER:string,OP_CARRIER_FL_NUM:int,ORIGIN:string,ORIGIN_CITY_NAME:string,DEST:string,DEST_CITY_NAME:string,CRS_DEP_TIME:int,DEP_TIME:int,WHEELS_ON:int,TAXI_IN:int,CRS_ARR_TIME:int,ARR_TIME:int,CANCELLED:int,DISTANCE:int>
+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+--------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|1/1/2000|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|1/1/2000|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA

In [35]:
df_json=spark.read.format("json").load('flight-time.json')
print("JSON Schema"+df_json.schema.simpleString())
df_json.show(5)

struct<ARR_TIME:bigint,CANCELLED:bigint,CRS_ARR_TIME:bigint,CRS_DEP_TIME:bigint,DEP_TIME:bigint,DEST:string,DEST_CITY_NAME:string,DISTANCE:bigint,FL_DATE:string,OP_CARRIER:string,OP_CARRIER_FL_NUM:bigint,ORIGIN:string,ORIGIN_CITY_NAME:string,TAXI_IN:bigint,WHEELS_ON:bigint,_corrupt_record:string>
+--------+---------+------------+------------+--------+----+--------------+--------+--------+----------+-----------------+------+----------------+-------+---------+---------------+
|ARR_TIME|CANCELLED|CRS_ARR_TIME|CRS_DEP_TIME|DEP_TIME|DEST|DEST_CITY_NAME|DISTANCE| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|TAXI_IN|WHEELS_ON|_corrupt_record|
+--------+---------+------------+------------+--------+----+--------------+--------+--------+----------+-----------------+------+----------------+-------+---------+---------------+
|    1348|        0|        1400|        1115|    1113| ATL|   Atlanta, GA|     946|1/1/2000|        DL|             1451|   BOS|      Boston, MA|      5|     

In [14]:
df_parqet=spark.read.format("parquet").load('flight-time.parquet')
print("parquet Schema"+df_parqet.schema.simpleString())
df_parqet.show(5)

parquet Schemastruct<FL_DATE:date,OP_CARRIER:string,OP_CARRIER_FL_NUM:int,ORIGIN:string,ORIGIN_CITY_NAME:string,DEST:string,DEST_CITY_NAME:string,CRS_DEP_TIME:int,DEP_TIME:int,WHEELS_ON:int,TAXI_IN:int,CRS_ARR_TIME:int,ARR_TIME:int,CANCELLED:int,DISTANCE:int>
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|  

### set schema explicitly 

In [4]:
from pyspark.sql.types import StructType, StructField, DateType, StringType, IntegerType
flightSchemaStruct = StructType([
    StructField("FL_DATE", DateType()),
    StructField("OP_CARRIER", StringType()),
    StructField("OP_CARRIER_FL_NUM", IntegerType()),
    StructField("ORIGIN", StringType()),
    StructField("ORIGIN_CITY_NAME", StringType()),
    StructField("DEST", StringType()),
    StructField("DEST_CITY_NAME", StringType()),
    StructField("CRS_DEP_TIME", IntegerType()),
    StructField("DEP_TIME", IntegerType()),
    StructField("WHEELS_ON", IntegerType()),
    StructField("TAXI_IN", IntegerType()),
    StructField("CRS_ARR_TIME", IntegerType()),
    StructField("ARR_TIME", IntegerType()),
    StructField("CANCELLED", IntegerType()),
    StructField("DISTANCE", IntegerType())
])

In [41]:
df_csv=spark.read.format("csv").option("header","true").schema(flightSchemaStruct).option("mode","FAILFAST").option("dateFormat","M/d/y").load('flight-time.csv')
print("CSV Schema" +df_csv.schema.simpleString())
df_csv.show(5)

CSV Schemastruct<FL_DATE:date,OP_CARRIER:string,OP_CARRIER_FL_NUM:int,ORIGIN:string,ORIGIN_CITY_NAME:string,DEST:string,DEST_CITY_NAME:string,CRS_DEP_TIME:int,DEP_TIME:int,WHEELS_ON:int,TAXI_IN:int,CRS_ARR_TIME:int,ARR_TIME:int,CANCELLED:int,DISTANCE:int>
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atl

In [43]:
df_json=spark.read.format("json").schema(flightSchemaStruct).option("mode","FAILFAST").option("dateFormat","M/d/y").load('flight-time.json')
print("JSON Schema"+df_json.schema.simpleString())
df_json.show(5)

JSON Schemastruct<FL_DATE:date,OP_CARRIER:string,OP_CARRIER_FL_NUM:int,ORIGIN:string,ORIGIN_CITY_NAME:string,DEST:string,DEST_CITY_NAME:string,CRS_DEP_TIME:int,DEP_TIME:int,WHEELS_ON:int,TAXI_IN:int,CRS_ARR_TIME:int,ARR_TIME:int,CANCELLED:int,DISTANCE:int>
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   At

### Dataframe Writer 

In [15]:
"""
DataFrame.write
.format("") parqet by default, json ...
.mode(saveMode) append,overwrite,ignore,errorifExists
.option()
.partitionBy()
.save()"""
df_parqet.write.format('json').mode('overwrite').option('path',r'C:\Users\kkirugulige\OneDrive\OneDrive - HARMAN\Jupyter\New folder\\').save()

Py4JJavaError: An error occurred while calling o57.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:

In [16]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()


In [17]:
df_parqet.write.mode('overwrite').saveAsTable('Flight_data_tbl')

Py4JJavaError: An error occurred while calling o64.saveAsTable.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSyste

### Transformation 

In [23]:
file_df = spark.read.text('apache_logs.txt')
file_df.printSchema()

root
 |-- value: string (nullable = true)



In [26]:
from pyspark.sql import *
from pyspark.sql.functions import regexp_extract, substring_index
log_reg = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\S+) "(\S+)" "([^"]*)'

logs_df = file_df.select(regexp_extract('value', log_reg, 1).alias('ip'),
                         regexp_extract('value', log_reg, 4).alias('date'),
                         regexp_extract('value', log_reg, 6).alias('request'),
                         regexp_extract('value', log_reg, 10).alias('referrer'))
logs_df.printSchema()

logs_df \
    .where("trim(referrer) != '-' ") \
    .withColumn("referrer", substring_index("referrer", "/", 3)) \
    .groupBy("referrer") \
    .count() \
    .show(100, truncate=False)

root
 |-- ip: string (nullable = true)
 |-- date: string (nullable = true)
 |-- request: string (nullable = true)
 |-- referrer: string (nullable = true)

+--------------------------------------+-----+
|referrer                              |count|
+--------------------------------------+-----+
|http://ijavascript.cn                 |1    |
|http://www.google.co.tz               |1    |
|http://www.google.ca                  |6    |
|https://www.google.hr                 |2    |
|https://www.google.ch                 |1    |
|http://www.google.ru                  |6    |
|http://www.raspberrypi-spanish.es     |1    |
|http://semicomplete.com               |2001 |
|http://manpages.ubuntu.com            |2    |
|http://kufli.blogspot.fr              |1    |
|http://www.bing.com                   |6    |
|http://rungie.com                     |1    |
|http://www.google.co.th               |2    |
|https://www.google.cz                 |5    |
|http://danceuniverse.ru               |3    |

In [30]:
logs_df.select(to_date(logs_df.date)).show(5)

NameError: name 'to_date' is not defined

In [31]:
logs_df.show(5)

+------------+--------------------+--------------------+--------------------+
|          ip|                date|             request|            referrer|
+------------+--------------------+--------------------+--------------------+
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
+------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

### UDF Demo

In [2]:
gender_df = spark.read.format("csv").option("header","true").option('inferSchema','true').csv('gender.csv')
gender_df.show(10)

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+----------+-----------------------+---------------------+------------------+---------------+--------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+--

In [7]:
import re
def parse_gender(gender):
    male_pat= r"m$|m.a|m.l"
    female_pat = r"^f$|f.m|w.m"
    if re.search(female_pat,gender.lower()):
        return "Female"
    elif re.search(male_pat,gender.lower()):
        return "Male"
    else:
        return "UNKNOWN"

In [8]:
#register the udf function
parse_gender_udf =udf(parse_gender,StringType())
gender_df2 =gender_df.withColumn("Gender",parse_gender_udf("Gender"))
gender_df2.show(10)

Py4JJavaError: An error occurred while calling o84.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5) (INBAWN162959.ad.harman.com executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=1, Incorrect function
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1142)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1073)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:167)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.Ev

In [9]:
invoice_df = spark.read \
.format("csv") \
.option("header", "true") \
.option("inferSchema", "true") \
.load("invoices.csv")

In [15]:
from pyspark.sql import functions as f
invoice_df.select(f.count("*").alias("Count *"),
f.sum("Quantity").alias("TotalQuantity"),
f.avg("UnitPrice").alias("AvgPrice"),
f.countDistinct("InvoiceNo").alias("CountDistinct")
).show()

#pass sql like expr
invoice_df.selectExpr("count(1) as `count`",
                     "sum(Quantity) as TotalQuantity",
                     "avg(UnitPrice) as AvgPrice").show()

+-------+-------------+-----------------+-------------+
|Count *|TotalQuantity|         AvgPrice|CountDistinct|
+-------+-------------+-----------------+-------------+
| 133577|      1184946|5.191259872582844|         6397|
+-------+-------------+-----------------+-------------+

+------+-------------+-----------------+
| count|TotalQuantity|         AvgPrice|
+------+-------------+-----------------+
|133577|      1184946|5.191259872583614|
+------+-------------+-----------------+



In [26]:
#pass sql like expr
invoice_df.createOrReplaceTempView("sales")
summary_sql = spark.sql("""Select Country,InvoiceNo, sum(Quantity) as TotQuantity, round(sum(Quantity *UnitPrice),2) as InvoiceValue
from sales
GROUP BY Country,InvoiceNo""")
summary_sql.show()

#use df exprs
summary_df = invoice_df \
    .groupBy("Country","InvoiceNo").agg(f.sum("Quantity").alias("TotQuantity"),f.round(f.sum(f.expr("Quantity * Unitprice")),2).alias("InvoiceValue")).show()
    
    

+--------------+---------+-----------+------------+
|       Country|InvoiceNo|TotQuantity|InvoiceValue|
+--------------+---------+-----------+------------+
|United Kingdom|   536446|        329|      440.89|
|United Kingdom|   536508|        216|      155.52|
|United Kingdom|   537018|         -3|         0.0|
|United Kingdom|   537401|        -24|         0.0|
|United Kingdom|   537811|         74|      268.86|
|United Kingdom|  C537824|         -2|       -14.9|
|United Kingdom|   538895|        370|      247.38|
|United Kingdom|   540453|        341|      302.45|
|United Kingdom|   536627|         64|       306.2|
|United Kingdom|   537224|        700|     1415.97|
|United Kingdom|   537230|          1|        2.95|
|United Kingdom|   537682|         72|        99.6|
|United Kingdom|  C538749|         -1|       -9.95|
|        Israel|  C539037|        -56|     -227.44|
|United Kingdom|   539076|         96|       391.0|
|United Kingdom|   539453|       1121|     5022.16|
|United King

In [32]:
#create a new column where you get the week 
n_invoice_df=invoice_df.withColumn("InvoiceDate",f.to_date(f.col("InvoiceDate"),"dd-MM-yyyy H.mm")) \
                       .withColumn("WeekNumber",f.weekofyear(f.col("InvoiceDate"))) \
#summary_df = n_invoice_df \
 #   .groupBy("Country","InvoiceNo").agg(f.sum("Quantity").alias("TotQuantity"),f.round(f.sum(f.expr("Quantity * Unitprice")),2).alias("InvoiceValue")).show()
    
    

n_invoice_df.createOrReplaceTempView("salesweek")
summary_sql = spark.sql("""Select Country,WeekNumber, count(InvoiceNo) as NumInvoices,sum(Quantity) as TotQuantity, round(sum(Quantity *UnitPrice),2) as InvoiceValue
from salesweek
GROUP BY Country,WeekNumber
ORDER BY Country,WeekNumber """)
summary_sql.show()

+---------+----------+-----------+-----------+------------+
|  Country|WeekNumber|NumInvoices|TotQuantity|InvoiceValue|
+---------+----------+-----------+-----------+------------+
|Australia|         1|         48|       4802|     7154.38|
|Australia|         2|         23|        394|       721.1|
|Australia|         3|         36|        338|      757.55|
|Australia|         4|         20|        110|      384.68|
|Australia|         6|         12|         58|       192.0|
|Australia|         7|         69|       8384|    14022.92|
|Australia|         8|         10|        217|      412.55|
|Australia|         9|         84|      10162|    16558.14|
|Australia|        10|         10|         90|       210.9|
|Australia|        12|         16|         77|      286.25|
|Australia|        48|         14|        107|      358.25|
|Australia|        49|          8|        214|       258.9|
|Australia|        50|         13|        133|      387.95|
|  Austria|         5|         21|      

# Create managed table

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [5]:
flight_df = spark.read \
    .format('parquet') \
    .load('flight-time.parquet')

In [7]:
flight_df.show(5)

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

In [9]:
spark.sql("CREATE DATABASE IF NOT EXISTS ARLINE_DB")
flight_df.write \
    .mode('overwrite') \
    .saveAsTable('ARLINE_DB.flight_tbl_data')

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: MetaException(message:java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems)

# UDF function

In [12]:
#import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [15]:
def to_date_df(df,fmt,fld):
    return df.withColumn(fld,to_date(col(fld),fmt))

In [16]:
if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .master("local[3]") \
        .appName("RowDemo") \
        .getOrCreate()

In [17]:
my_schema = StructType([
    StructField("ID", StringType()),
    StructField("EventDate", StringType())])

my_rows = [Row("123", "04/05/2020"), Row("124", "4/5/2020"), Row("125", "04/5/2020"), Row("126", "4/05/2020")]
my_rdd = spark.sparkContext.parallelize(my_rows, 2)
my_df = spark.createDataFrame(my_rdd, my_schema)
print(my_df.printSchema())
print(my_df.show(5))
new_df = to_date_df(my_df,"M/d/y","EventDate")

root
 |-- ID: string (nullable = true)
 |-- EventDate: string (nullable = true)

None


Py4JJavaError: An error occurred while calling o66.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (INBAWN162959.ad.harman.com executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrRe

In [ ]:
print(new_df.printSchema())
print(new_df.show(5))

In [ ]:
new_df.show()